In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms

IMAGENET_MEAN = [0.485, 0.456, 0.406]  # Mean for R, G, B
IMAGENET_STD = [0.229, 0.224, 0.225]   # Std Dev for R, G, B

data_transforms = transforms.Compose([
     transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

training_data = CIFAR10(
    root="../data", 
    train=True,
    download=True,
    transform=data_transforms
)

testing_data = CIFAR10(
    root = "../data",
    train= False,
    download=True,
    transform=test_transforms
)
print(training_data)

batch_size = 64
train_dataloader = DataLoader(
    dataset=training_data,
    batch_size=batch_size,
    shuffle=True
)
test_dataloader = DataLoader(
    dataset=testing_data,
    batch_size=batch_size,
    shuffle=False
)

images, labels = next(iter(train_dataloader))
print(images.shape, labels.shape)


/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/jtappen/.local/share/mamba/envs/ml/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can igno

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
torch.Size([64, 3, 32, 32]) torch.Size([64])


In [3]:
class SimpleConvNet(nn.Module):
    def __init__(self, input_channels=3, num_classes=10):
        super(SimpleConvNet,  self).__init__()
        self.layers = nn.Sequential(
            # Block 1
            nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding = 1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3 , stride=2, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Block 2
            nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding= 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Classification Head
            nn.Flatten(),
            nn.Linear(128 * 2 *2, num_classes)
        )

    def forward(self, X):
        return self.layers(X)


In [4]:
# Step 3: Inititalize Model, Optimizer, Loss Function
model = SimpleConvNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Step 4: Training Loop
epochs = 10
model.train()


for i in range(epochs):
    avg_loss = 0
    num_batches = len(train_dataloader)
    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()

        logits = model(X)
        loss = loss_fn(logits, y)

        loss.backward()

        optimizer.step()

        avg_loss += loss.item()

    print(f"Epoc {i+1}----------------------------------")
    print(f"Average Loss: {avg_loss/num_batches}")

print("Completed Training")





Epoc 1----------------------------------
Average Loss: 0.8877639306323303
Epoc 2----------------------------------
Average Loss: 0.8756545419278352
Epoc 3----------------------------------
Average Loss: 0.8581701567791917
Epoc 4----------------------------------
Average Loss: 0.837968643935745
Epoc 5----------------------------------
Average Loss: 0.8319406456044872
Epoc 6----------------------------------
Average Loss: 0.8133045871315709
Epoc 7----------------------------------
Average Loss: 0.8035156596118532
Epoc 8----------------------------------
Average Loss: 0.7934937552570382
Epoc 9----------------------------------
Average Loss: 0.7829759532152234
Epoc 10----------------------------------
Average Loss: 0.777169937634712
Completed Training


In [13]:
num_batches = len(train_dataloader)
total_accuracy = 0
test_loss = 0

for X,y in train_dataloader:
    model.eval()
    with torch.no_grad():
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = loss_fn(logits, y)

        test_loss += loss.item()

        pred_classes= torch.argmax(logits, dim=1)
        batch_correct = (pred_classes == y).sum().item()
        total_accuracy += (batch_correct / y.size(0))


print(f"Results:")
print(f"Test Loss: {test_loss/num_batches}")
print(f"Total Accuracy: {total_accuracy/num_batches}")




        


Results:
Test Loss: 0.7347688286200814
Total Accuracy: 0.7391104539641944
